# Example Google Earth Engine notebook


In [3]:
#!pip install earthengine-api

     ---------------------------------------- 2.2/2.2 MB 19.7 MB/s eta 0:00:00
     ---------------------------------------- 98.6/98.6 kB ? eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 41.1 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 39.9 MB/s eta 0:00:00
     ---------------------------------------- 102.3/102.3 kB ? eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 32.3 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 130.5/130.5 kB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB ? eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=b7ff3262bb2da5d86752a50b49ee00b3a7a1ca28543584757b53c8c0403e3859
  Stored in directory: c:\users\efqyx\appdata\

In [1]:
# pre-install and import all the libraries in one go
import numpy as np
import ee   
import numpy as np
import os
import time
import pandas as pd

# EE Python API

In [2]:
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [3]:
ee.data.setDefaultWorkloadTag('ee-gee-test-notebook-evandelancey')

In [4]:
ic = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate('2021-01-01', '2021-01-10')
print(ic.size().getInfo())

78521


In [5]:
im = ic.first()
print(im.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


# Gee Map

In [7]:
#Test accessing data from Bayer GEE assests
import geemap
myMap = geemap.Map()
clusters = ee.Image('projects/bcs-loc360-earth-engine-dev/assets/vegClustering/vegClusters_testRun')
myMap.addLayer(clusters.randomVisualizer(), {}, 'Veg clusters')
myMap.centerObject(clusters)
myMap

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Common GEE datasets

## Sentinel-2 optical

In [8]:
#test geometry
geom = ee.Geometry.BBox(-120.9196, 36.4467, -119.4035, 37.3909)
s2_ic = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate('2023-04-01', '2023-04-30').filterBounds(geom)
s2_im = s2_ic.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).median()

s2Map = geemap.Map()
s2Map.centerObject(geom)
s2Map.addLayer(s2_im, {'min':0, 'max':3000, 'bands': ['B4', 'B3', 'B2']}, 'April 2023 Sentinel-2')
s2Map.addLayer(geom, {"color": '#00000000'}, 'geometry')
s2Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Sentinel-1 (SAR) 

In [9]:
s1_ic = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2023-04-01', '2023-04-30').filterBounds(geom)
s1_im = s1_ic.mean()

s1Map = geemap.Map()
s1Map.centerObject(geom)
s1Map.addLayer(s1_im, {'min':-20, 'max':0, 'bands': ['VV']}, 'April 2023 Sentinel-1 VV')
s1Map.addLayer(geom, {"color": '#00000000'}, 'geometry')
s1Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## weather (Daymet v4)

In [10]:
daymet_ic = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate('2021-04-01', '2021-04-02')

temp = daymet_ic.select('tmax').mean()

tempMap = geemap.Map()
tempMap.centerObject(geom)
tempMap.addLayer(temp, {'min':10, 'max':32, 'palette': ['blue', 'yellow', 'red']}, 'April 1st temp')
tempMap.addLayer(geom, {"color": '#00000000'}, 'geometry')
tempMap

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## soil data (soilgrids)

In [11]:
soilNit = ee.Image("projects/soilgrids-isric/nitrogen_mean").select('nitrogen_0-5cm_mean').divide(100)
soilMap = geemap.Map()
soilMap.centerObject(geom)
soilPal = ['#fff5eb','#fee6ce','#fdd0a2','#fdae6b','#fd8d3c','#f16913','#d94801','#a63603','#7f2704']
soilMap.addLayer(soilNit, {'min':0, 'max':35, 'palette': soilPal}, 'Soil nitrogen')
soilMap.addLayer(geom, {"color": '#00000000'}, 'geometry')
soilMap

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…